## protocol:
* get cigar
* get soft-clipped sequence incl gatc site
* blat soft-clipped sequence

In [59]:
def getSoftSigar(cigar, fields):
    seq = ''
    qual = ''
    S = re.split('(\d+)',cigar)
    S = filter(None, S)
    idx = S.index("S")
    if idx == 1:
        se = fields[9][   0  :  int(S[0])+4   ]
        que = fields[10][   0  :  int(S[0])+4   ]
        if 'GATC' in se:
            GATCstart = se.index('GATC')
            seq = se[:GATCstart+4]
            qual = que[:GATCstart+4]
    elif idx == len(S)-1:
        S5 = False
        se = fields[9][((len(fields[9]))-int(S[len(S)-2]))-4:(len(fields[9]))] 
        qua = fields[10][((len(fields[10]))-int(S[len(S)-2]))-4:(len(fields[10]))] 
        if 'GATC' in se:
            GATCstart = se.index('GATC')
            seq = se[GATCstart:]
            qual = qua[GATCstart:]
    return [seq,qual]

def extractXAcigars(fields):
    XA = []
    for idx,entry in enumerate(fields):
        if 'XA:' in entry:
            XAlist = entry[5:].split(';')
            for i in XAlist:
                xa = i.split(',')
                if len(xa) > 1:
                    XA.append(xa)
            XA = filter(None, XA)
    return(XA)

def ReverseComplement1(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

In [73]:
import re
samInput = open('/Users/robin/GitHub/SLR-HiC/data/BC001_sorted.sam','r')

fqOut = open('/Users/robin/GitHub/SLR-HiC/data/BC001_sorted_softclipped.fq','w')

for e,line in enumerate(samInput):
    if 'XA:' in line: # this is a read we can use!
        fields = line.rstrip().split("\t")
        cigar = fields[5]
        orientation = "+"
        if 'S' not in cigar:
            continue
        if 'GATC' not in fields[9]:
            continue
        if fields[1] == 16:
            orientation = "-"
        seq,qual = getSoftSigar(cigar,fields)
        if orientation == "-":
            seq = ReverseComplement1(seq)
        if len(qual) > 1:
            header = str("@") + ":".join([fields[0],fields[2],fields[3]])
            fqOut.write(header + "\n")
            fqOut.write( seq+ "\n")
            fqOut.write( "+"+ "\n")
            fqOut.write( qual+ "\n")
fqOut.close()